In [4]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
from selenium import webdriver
#from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
import dateparser
import os

def scrap_novojob():
    # Liste des liens pour chaque catégorie
    categories = [
        "toutes les offres d'emploi",
        
    ]

    base_url = "https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q="
    category_links = [f"{base_url}{'+'.join(category.split(','))}" for category in categories]

    # Utilisation d'un en-tête pour éviter d'être bloqué
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    intitules_list = []
    entreprises_list = []
    pays_list = []
    dates_list = []
    niveau_list = []
    experience_list = []

    url_list = []
    all_job_lien = []

    # Parcourir les liens de chaque catégorie
    for category_link in category_links:
        req = requests.get(category_link, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')
        time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

        offres = soup.find_all('div', class_='row-fluid job-details pointer')

        for offre in offres:
            offre_link_tag = offre.find('a')
            if offre_link_tag:
                offre_link = offre_link_tag['href']
                all_job_lien.append(offre_link)

            entreprise_tag = offre.find('h6', class_='ellipsis')
            entreprise = entreprise_tag.get_text().strip() if entreprise_tag else None

            intitule_tag = offre.find('h2', class_='ellipsis row-fluid')
            intitule = intitule_tag.get_text().strip() if intitule_tag else None

            bloc_bottom = offre.find_next('div', class_='bloc-bottom')

            pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
            pays = pays_info.find_parent().text.strip() if pays_info else None

            date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
            date = date_info.find_parent().text.strip() if date_info else None

            niveau_info = bloc_bottom.find('i', class_='fa fa-bookmark icon-left')
            niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

            match = re.match(r'(.+) \((.+)\)', niveau_text)
            niveau_col, experience_col = match.groups() if match else (None, None)

            intitules_list.append(intitule)
            entreprises_list.append(entreprise)
            pays_list.append(pays)
            dates_list.append(date)
            niveau_list.append(niveau_col)
            experience_list.append(experience_col)
            
            url_list.append(category_link)

    # Création du DataFrame
    df_offers = pd.DataFrame({
        'Intitule': intitules_list,
        'Entreprise': entreprises_list,
        'Pays': pays_list,
        'Date': dates_list,
        'Niveau': niveau_list,
        'Experience_lettre': experience_list,
        'url': url_list,
        'Offre_Link': all_job_lien
    })

    # Importer les détails de chaque offre d'emploi
    job_urls = list(df_offers['Offre_Link'])
    all_job_details = []

    for url in job_urls:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }

        req = requests.get(url, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')

        job_details = {}
        
        # Ajouter le lien
        job_details["Offre_Link"] = url
        
        # Extraction des détails de l'offre d'emploi
        details_section = soup.find('ul', class_='text-small')
        if details_section:
            for li in details_section.find_all('li', class_='row-fluid'):
                key = li.find('span', class_='span4').text.strip()
                value = li.find('span', class_='span8').text.strip()
                job_details[key] = value

        # Extraction du texte fourni
        description_section = soup.find('div', class_='spaced details-description')
        if description_section:
            provided_text = description_section.text.strip()
            job_details['Provided Text'] = provided_text

        all_job_details.append(job_details)

    # Création d'un DataFrame pour les détails des offres d'emploi
    df_Novojob = pd.DataFrame(all_job_details)
    # Ajouter les listes existantes en tant que colonnes au DataFrame
    df_Novojob = pd.merge(df_offers,df_Novojob, on='Offre_Link')
    df_Novojob = df_Novojob.drop_duplicates()
    df_Novojob.reset_index(drop=True, inplace=True)

    
    equivalences = {
    "Offre_Link": "SITE_WEB_DE_L_ENTREPRISE",
    "Nom de l'entreprise": "RAISON_SOCIALE_DE_L_ENTREPRISE",
    "Secteur d'activité": "BRANCHE_D_ACTIVITE",
    "Lieu de travail": "LIEU_DU_POSTE_DE_TRAVAIL",
    "Date d'expiration": "DATE_D_EXPIRATION_DE_L_OFFRE",
    "Nombre de postes": "NOMBRE_DE_POSTES_A_POURVOIR",
    "Niveau de poste": None,
    "Niveau d'étude (diplôme)": "NIVEAU_D_ETUDES",
    "Type de contrat": "TYPE_DE_CONTRAT_DU_POSTE",
    "Provided Text": "Texte_fourni",
    "Intitule": "INTITULE_DU_POSTE",
    "Entreprise": None,
    "Pays": "PAYS_DU_POSTE_DE_TRAVAIL",
    "Date": "DATE_DE_DEBUT_DE_L_OFFRE",
    "Niveau": None,
    "Experience_lettre": None,
    "url_Lien": "Offre_Link1"}
    # Fonction pour renommer les colonnes du DataFrame en conservant les colonnes sans équivalence
    def renommer_colonnes(df, equivalences):
        colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if nouvel_nom is not None}
        df_renomme = df.rename(columns=colonnes_renommees)
        return df_renomme
    df_Novojob = renommer_colonnes(df_Novojob, equivalences)
    df_Novojob['NOMBRE_DE_POSTES_A_POURVOIR'] = df_Novojob['NOMBRE_DE_POSTES_A_POURVOIR'].str.extract(r"([0-9,]+)")
    df_Novojob[["VILLE_DU_POSTE_DE_TRAVAIL","PAYS_DU_POSTE_DE_TRAVAIL"]] = df_Novojob['LIEU_DU_POSTE_DE_TRAVAIL'].str.split(',', expand=True)
    for i in range(len(df_Novojob["PAYS_DU_POSTE_DE_TRAVAIL"])):
        if df_Novojob["PAYS_DU_POSTE_DE_TRAVAIL"][i] is None :
            df_Novojob.loc[i, "PAYS_DU_POSTE_DE_TRAVAIL"] = df_Novojob["VILLE_DU_POSTE_DE_TRAVAIL"][i]
            df_Novojob.loc[i, "VILLE_DU_POSTE_DE_TRAVAIL"]= " "


    # Réorganiser les colonnes selon vos besoins

    return df_Novojob

df_novojob=scrap_novojob()
df_novojob1=df_novojob
df_novojob

,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,Niveau,Experience_lettre,url,SITE_WEB_DE_L_ENTREPRISE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,Niveau de poste,BRANCHE_D_ACTIVITE,Texte_fourni,Niveau d'étude (diplome),NOMBRE_DE_POSTES_A_POURVOIR,TYPE_DE_CONTRAT_DU_POSTE,RAISON_SOCIALE_DE_L_ENTREPRISE,VILLE_DU_POSTE_DE_TRAVAIL
0,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,Manager / Responsable département,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,Manager / Responsable département,Services,Entreprise spécialisée dans l’ingénierie mécan...,NaN,NaN,NaN,NaN,Abidjan
1,Commercial Terrain,KOMIAN AI,Côte d'ivoire,02 Février,Débutant / Junior,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,Aujourd'hui,Débutant / Junior| Stagiaire / Etudiant| Jeune...,"Informatique, Télécom, Internet",KOMIAN IA est une entreprise de technologie d’...,"TS Bac +2| Licence (LMD), Bac + 3",25,Consultant,NaN,
2,Stagiaire Réseau Fibre Optique et Informatique,WANDOO TECHNOLOGIES,Côte d'ivoire,25 Avril,Stagiaire / Etudiant,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,24 Juillet,Stagiaire / Etudiant,"Informatique, Télécom, Internet",Description de la mission Participation à la r...,NaN,01,Stage – Temps partiel,NaN,
3,Chiffreur BTP,Exceliam,Côte d'ivoire,08 Février,Responsable d'équipe,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,Responsable d'équipe,Services,Sous la responsabilité du Responsable Techniqu...,"Master 2, Ingéniorat, Bac + 5",01,CDI,NaN,Abidjan
4,Chargé Administratif RH et Commercial Adjoint,Exceliam,Côte d'ivoire,07 Février,Responsable d'équipe,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",05 Mai,Responsable d'équipe| Confirmé / Expérimenté,Services,Sous le contrôle direct du Responsable de l’Ad...,"Licence (LMD), Bac + 3| Master 1, Licence Bac...",01,CDI| CDD,NaN,Abidjan
5,Chargé D'Affaires Senior,Exceliam,Côte d'ivoire,08 Février,Responsable d'équipe,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,Responsable d'équipe,Services,Sous la responsabilité du Responsable Techniqu...,"Licence (LMD), Bac + 3| Master 1, Licence Bac...",02,CDI,NaN,Abidjan
6,Assistante Anglais et Français pour un Salon d...,FutureTech,Côte d'ivoire,04 Mars,Débutant / Junior,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juin,Débutant / Junior,"Informatique, Télécom, Internet",Dans le cadre de notre participation à un salo...,NaN,01,NaN,NaN,
7,Télévendeur H/F,OLSY GROUP,Côte d'ivoire,03 Avril,Confirmé / Expérimenté,1 à 2 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juillet,Débutant / Junior| Confirmé / Expérimenté,Services,OLSY GROUP est une entreprise qui offre des se...,TS Bac +2,10,CDD,NaN,
8,Responsable Commercial,Exceliam,Côte d'ivoire,06 Mars,Manager / Responsable département,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",04 Juin,Responsable d'équipe| Manager / Responsable dé...,Services,Nous recherchons pour notre client leader dans...,"Licence (LMD), Bac + 3| Master 1, Licence Bac...",01,CDI| CDD,NaN,Abidjan
9,Démonstrateur,Entreprise anonyme,Côte d'ivoire,23 Février,Confirmé / Expérimenté,3 à 5 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",23 

In [31]:
df_final = pd.concat([df_novojob,df_novojob1 ], axis=0, ignore_index=True)
df_final

,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,Niveau,Experience_lettre,url,SITE_WEB_DE_L_ENTREPRISE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,Niveau de poste,BRANCHE_D_ACTIVITE,Texte_fourni,Niveau d'étude (diplome),NOMBRE_DE_POSTES_A_POURVOIR,TYPE_DE_CONTRAT_DU_POSTE,RAISON_SOCIALE_DE_L_ENTREPRISE,VILLE_DU_POSTE_DE_TRAVAIL
0,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,Manager / Responsable département,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,Manager / Responsable département,Services,Entreprise spécialisée dans l’ingénierie mécan...,NaN,NaN,NaN,NaN,Abidjan
1,Commercial Terrain,KOMIAN AI,Côte d'ivoire,02 Février,Débutant / Junior,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,Aujourd'hui,Débutant / Junior| Stagiaire / Etudiant| Jeune...,"Informatique, Télécom, Internet",KOMIAN IA est une entreprise de technologie d’...,"TS Bac +2| Licence (LMD), Bac + 3",25,Consultant,NaN,
2,Stagiaire Réseau Fibre Optique et Informatique,WANDOO TECHNOLOGIES,Côte d'ivoire,25 Avril,Stagiaire / Etudiant,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,24 Juillet,Stagiaire / Etudiant,"Informatique, Télécom, Internet",Description de la mission Participation à la r...,NaN,01,Stage – Temps partiel,NaN,
3,Chiffreur BTP,Exceliam,Côte d'ivoire,08 Février,Responsable d'équipe,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,Responsable d'équipe,Services,Sous la responsabilité du Responsable Techniqu...,"Master 2, Ingéniorat, Bac + 5",01,CDI,NaN,Abidjan
4,Chargé Administratif RH et Commercial Adjoint,Exceliam,Côte d'ivoire,07 Février,Responsable d'équipe,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",05 Mai,Responsable d'équipe| Confirmé / Expérimenté,Services,Sous le contrôle direct du Responsable de l’Ad...,"Licence (LMD), Bac + 3| Master 1, Licence Bac...",01,CDI| CDD,NaN,Abidjan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Chef de Projet LAN / WLAN,Ascens,Côte d'ivoire,02 Février,Confirmé / Expérimenté,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",Aujourd'hui,Confirmé / Expérimenté,"Informatique, Télécom, Internet",Présentation :ASCENS Services est une filiale ...,"Master 1, Licence Bac + 4| Master 2, Ingénior...",01,Consultant,NaN,Abidjan
96,Ingénieur Systèmes Informatique,Prosuma,Côte d'ivoire,02 Avril,Confirmé / Expérimenté,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",01 Juillet,Confirmé / Expérimenté,"Distribution, Commerce",Définition L’Ingénieur Systèmes Informatique ...,"Master 2, Ingéniorat, Bac + 5",02,CDI| CDD,NaN,Abidjan
97,Stagiaire professionnel(le) Assistant(e) Compt...,Entreprise anonyme,Côte d'ivoire,22 Février,Débutant / Junior,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Mai,Débutant / Junior,"BTP, Construction, Immobilier",Présentation de l’entreprise Intervenant da...,TS Bac +2,01,Pré-emploi,Entreprise anonyme,Abidjan
98,Responsable du Pilotage commercial Retail AFS,Société Générale Afrique de l'ouest ( Centre ...,Côte d'ivoire,08 Avril,Manager / Responsable département,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",07 Juillet,Manager / Responsable département| Confirmé / ...,"Banque, Assurance, Finance",Nous recrutons pour le Hub Market

In [53]:
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

def preprocess_text(text):
    # Convertir le texte en minuscules
    text = text.lower()
    # Enlever la ponctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

# Prétraitement des données
df_final['Texte_clean'] = df_final['Texte_fourni'].apply(preprocess_text)

# Vectorisation des données
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_final['Texte_clean'])

# Calcul de similarité
similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Seuil de similarité
threshold = 0.9

# Identification des doublons
duplicates = set()
for i in range(len(similarities)):
    for j in range(i+1, len(similarities)):
        if similarities[i,j] > threshold:
            duplicates.add(j)

# Suppression des doublons
df_final = df_final.drop(index=duplicates).reset_index(drop=True)

# Afficher ou retourner le DataFrame final
df_final


,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,Niveau,Experience_lettre,url,SITE_WEB_DE_L_ENTREPRISE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,...,NOMBRE_DE_POSTES_A_POURVOIR,TYPE_DE_CONTRAT_DU_POSTE,RAISON_SOCIALE_DE_L_ENTREPRISE,VILLE_DU_POSTE_DE_TRAVAIL,INTITULE_DU_POSTE_clean,Texte_clean,Entreprise_clean,Niveau_etude_clean,TYPE_DE_CONTRAT_DU_POSTE_clean,Combined_text
0,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,Manager / Responsable département,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,...,NaN,NaN,NaN,Abidjan,gestionnaire des ressources humaines,entreprise spécialisée dans l’ingénierie mécan...,,,,gestionnaire des ressources humaines entrepris...
1,Commercial Terrain,KOMIAN AI,Côte d'ivoire,02 Février,Débutant / Junior,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,Aujourd'hui,...,25,Consultant,NaN,,commercial terrain,komian ia est une entreprise de technologie d’...,komian ai,ts bac 2 licence lmd bac 3,consultant,commercial terrain komian ia est une entrepris...
2,Stagiaire Réseau Fibre Optique et Informatique,WANDOO TECHNOLOGIES,Côte d'ivoire,25 Avril,Stagiaire / Etudiant,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,24 Juillet,...,01,Stage – Temps partiel,NaN,,stagiaire réseau fibre optique et informatique,description de la mission participation à la r...,wandoo technologies,,stage – temps partiel,stagiaire réseau fibre optique et informatique...
3,Chiffreur BTP,Exceliam,Côte d'ivoire,08 Février,Responsable d'équipe,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,01,CDI,NaN,Abidjan,chiffreur btp,sous la responsabilité du responsable techniqu...,exceliam,master 2 ingéniorat bac 5,cdi,chiffreur btp sous la responsabilité du respon...
4,Chargé Administratif RH et Commercial Adjoint,Exceliam,Côte d'ivoire,07 Février,Responsable d'équipe,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",05 Mai,...,01,CDI| CDD,NaN,Abidjan,chargé administratif rh et commercial adjoint,sous le contrôle direct du responsable de l’ad...,exceliam,licence lmd bac 3 master 1 licence bac 4,cdi cdd,chargé administratif rh et commercial adjoint ...
5,Chargé D'Affaires Senior,Exceliam,Côte d'ivoire,08 Février,Responsable d'équipe,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,02,CDI,NaN,Abidjan,chargé daffaires senior,sous la responsabilité du responsable techniqu...,exceliam,licence lmd bac 3 master 1 licence bac 4,cdi,chargé daffaires senior sous la responsabilité...
6,Assistante Anglais et Français pour un Salon d...,FutureTech,Côte d'ivoire,04 Mars,Débutant / Junior,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juin,...,01,NaN,NaN,,assistante anglais et français pour un salon d...,dans le cadre de notre participation à un salo...,futuretech,,,assistante anglais et français pour un salon d...
7,Télévendeur H/F,OLSY GROUP,Côte d'ivoire,03 Avril,Confirmé / Expérimenté,1 à 2 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juillet,...,10,CDD,NaN,,télévendeur hf,olsy group est une entreprise qui offre des se...,olsy group,ts bac 2,cdd,télévendeur hf olsy group est une entreprise q...
8,Responsable Commercial,Exceliam,Côte d'ivoire,06 Mars,Manager / Responsable département,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte 

In [55]:
print(tfidf_matrix)

  (0, 22)	0.048083008621322004
  (0, 396)	0.033736425563249815
  (0, 1565)	0.03550175195556505
  (0, 791)	0.03758968991637744
  (0, 2332)	0.048083008621322004
  (0, 1069)	0.028560313016930742
  (0, 1746)	0.04014511945357671
  (0, 1551)	0.03550175195556505
  (0, 397)	0.048083008621322004
  (0, 1725)	0.048083008621322004
  (0, 266)	0.04343964112331034
  (0, 2773)	0.027563862787819762
  (0, 1692)	0.04014511945357671
  (0, 2444)	0.04343964112331034
  (0, 2925)	0.048083008621322004
  (0, 2896)	0.029651800748632155
  (0, 404)	0.0191584820436876
  (0, 1772)	0.048083008621322004
  (0, 622)	0.04343964112331034
  (0, 2172)	0.024269341118086138
  (0, 2700)	0.02115516889749287
  (0, 2320)	0.048083008621322004
  (0, 957)	0.04343964112331034
  (0, 938)	0.033736425563249815
  (0, 579)	0.04343964112331034
  :	:
  (47, 1171)	0.04169673426691654
  (47, 2102)	0.03747165151081233
  (47, 159)	0.03516717262883758
  (47, 1392)	0.09058042269244974
  (47, 1215)	0.02759829036027465
  (47, 901)	0.035167172628837

In [57]:
similarities.

(48, 48)

In [54]:
df_final

,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,Niveau,Experience_lettre,url,SITE_WEB_DE_L_ENTREPRISE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,...,NOMBRE_DE_POSTES_A_POURVOIR,TYPE_DE_CONTRAT_DU_POSTE,RAISON_SOCIALE_DE_L_ENTREPRISE,VILLE_DU_POSTE_DE_TRAVAIL,INTITULE_DU_POSTE_clean,Texte_clean,Entreprise_clean,Niveau_etude_clean,TYPE_DE_CONTRAT_DU_POSTE_clean,Combined_text
0,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,Manager / Responsable département,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,...,NaN,NaN,NaN,Abidjan,gestionnaire des ressources humaines,entreprise spécialisée dans l’ingénierie mécan...,,,,gestionnaire des ressources humaines entrepris...
1,Commercial Terrain,KOMIAN AI,Côte d'ivoire,02 Février,Débutant / Junior,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,Aujourd'hui,...,25,Consultant,NaN,,commercial terrain,komian ia est une entreprise de technologie d’...,komian ai,ts bac 2 licence lmd bac 3,consultant,commercial terrain komian ia est une entrepris...
2,Stagiaire Réseau Fibre Optique et Informatique,WANDOO TECHNOLOGIES,Côte d'ivoire,25 Avril,Stagiaire / Etudiant,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,24 Juillet,...,01,Stage – Temps partiel,NaN,,stagiaire réseau fibre optique et informatique,description de la mission participation à la r...,wandoo technologies,,stage – temps partiel,stagiaire réseau fibre optique et informatique...
3,Chiffreur BTP,Exceliam,Côte d'ivoire,08 Février,Responsable d'équipe,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,01,CDI,NaN,Abidjan,chiffreur btp,sous la responsabilité du responsable techniqu...,exceliam,master 2 ingéniorat bac 5,cdi,chiffreur btp sous la responsabilité du respon...
4,Chargé Administratif RH et Commercial Adjoint,Exceliam,Côte d'ivoire,07 Février,Responsable d'équipe,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",05 Mai,...,01,CDI| CDD,NaN,Abidjan,chargé administratif rh et commercial adjoint,sous le contrôle direct du responsable de l’ad...,exceliam,licence lmd bac 3 master 1 licence bac 4,cdi cdd,chargé administratif rh et commercial adjoint ...
5,Chargé D'Affaires Senior,Exceliam,Côte d'ivoire,08 Février,Responsable d'équipe,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,02,CDI,NaN,Abidjan,chargé daffaires senior,sous la responsabilité du responsable techniqu...,exceliam,licence lmd bac 3 master 1 licence bac 4,cdi,chargé daffaires senior sous la responsabilité...
6,Assistante Anglais et Français pour un Salon d...,FutureTech,Côte d'ivoire,04 Mars,Débutant / Junior,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juin,...,01,NaN,NaN,,assistante anglais et français pour un salon d...,dans le cadre de notre participation à un salo...,futuretech,,,assistante anglais et français pour un salon d...
7,Télévendeur H/F,OLSY GROUP,Côte d'ivoire,03 Avril,Confirmé / Expérimenté,1 à 2 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juillet,...,10,CDD,NaN,,télévendeur hf,olsy group est une entreprise qui offre des se...,olsy group,ts bac 2,cdd,télévendeur hf olsy group est une entreprise q...
8,Responsable Commercial,Exceliam,Côte d'ivoire,06 Mars,Manager / Responsable département,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte 

In [21]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

# Prétraitement des données
df_final['Texte_clean'] = df_final['Texte_fourni'].apply(preprocess_text)
df_final['INTITULE_DU_POSTE_clean'] = df_final['INTITULE_DU_POSTE'].apply(preprocess_text)

# Concaténer toutes les variables pour créer un texte combiné
df_final['Combined_text'] = df_final['INTITULE_DU_POSTE_clean']+ ' ' + df_final['Texte_clean'] 

# Vectorisation des données
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_final['Combined_text'])

# Calcul de similarité
similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Seuil de similarité
threshold = 0.9

# Identification des doublons
duplicates = set()
for i in range(len(similarities)):
    for j in range(i+1, len(similarities)):
        if similarities[i,j] > threshold:
            duplicates.add(j)

# Suppression des doublons
df_final = df_final.drop(index=duplicates).reset_index(drop=True)

# Afficher ou retourner le DataFrame final
df_final


,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,Niveau,Experience_lettre,url,SITE_WEB_DE_L_ENTREPRISE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,...,BRANCHE_D_ACTIVITE,Texte_fourni,Niveau d'étude (diplome),NOMBRE_DE_POSTES_A_POURVOIR,TYPE_DE_CONTRAT_DU_POSTE,RAISON_SOCIALE_DE_L_ENTREPRISE,VILLE_DU_POSTE_DE_TRAVAIL,Texte_clean,INTITULE_DU_POSTE_clean,Combined_text
0,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,Manager / Responsable département,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,...,Services,Entreprise spécialisée dans l’ingénierie mécan...,NaN,NaN,NaN,NaN,Abidjan,entreprise spécialisée dans l’ingénierie mécan...,gestionnaire des ressources humaines,gestionnaire des ressources humaines entrepris...
1,Commercial Terrain,KOMIAN AI,Côte d'ivoire,02 Février,Débutant / Junior,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,Aujourd'hui,...,"Informatique, Télécom, Internet",KOMIAN IA est une entreprise de technologie d’...,"TS Bac +2| Licence (LMD), Bac + 3",25,Consultant,NaN,,komian ia est une entreprise de technologie d’...,commercial terrain,commercial terrain komian ia est une entrepris...
2,Stagiaire Réseau Fibre Optique et Informatique,WANDOO TECHNOLOGIES,Côte d'ivoire,25 Avril,Stagiaire / Etudiant,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,24 Juillet,...,"Informatique, Télécom, Internet",Description de la mission Participation à la r...,NaN,01,Stage – Temps partiel,NaN,,description de la mission participation à la r...,stagiaire réseau fibre optique et informatique,stagiaire réseau fibre optique et informatique...
3,Chiffreur BTP,Exceliam,Côte d'ivoire,08 Février,Responsable d'équipe,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,Services,Sous la responsabilité du Responsable Techniqu...,"Master 2, Ingéniorat, Bac + 5",01,CDI,NaN,Abidjan,sous la responsabilité du responsable techniqu...,chiffreur btp,chiffreur btp sous la responsabilité du respon...
4,Chargé Administratif RH et Commercial Adjoint,Exceliam,Côte d'ivoire,07 Février,Responsable d'équipe,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",05 Mai,...,Services,Sous le contrôle direct du Responsable de l’Ad...,"Licence (LMD), Bac + 3| Master 1, Licence Bac...",01,CDI| CDD,NaN,Abidjan,sous le contrôle direct du responsable de l’ad...,chargé administratif rh et commercial adjoint,chargé administratif rh et commercial adjoint ...
5,Chargé D'Affaires Senior,Exceliam,Côte d'ivoire,08 Février,Responsable d'équipe,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,Services,Sous la responsabilité du Responsable Techniqu...,"Licence (LMD), Bac + 3| Master 1, Licence Bac...",02,CDI,NaN,Abidjan,sous la responsabilité du responsable techniqu...,chargé daffaires senior,chargé daffaires senior sous la responsabilité...
6,Assistante Anglais et Français pour un Salon d...,FutureTech,Côte d'ivoire,04 Mars,Débutant / Junior,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juin,...,"Informatique, Télécom, Internet",Dans le cadre de notre participation à un salo...,NaN,01,NaN,NaN,,dans le cadre de notre participation à un salo...,assistante anglais et français pour un salon d...,assistante anglais et français pour un salon d...
7,Télévendeur H/F,OLSY GROUP,Côte d'ivoire,03 Avril,Confirmé / Expérimenté,1 à 2 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres

In [35]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

def preprocess_text(text):
    if isinstance(text, str):  # Vérifier si le texte est une chaîne de caractères
        text = text.lower()
        text = text.translate(str.maketrans('', '', string.punctuation))
    elif isinstance(text, float):  # Si c'est une valeur float, remplacer par une chaîne vide
        text = ''
    return text

# Prétraitement des données
# Remplacer les valeurs manquantes par une chaîne vide dans les colonnes utilisées
df_final['INTITULE_DU_POSTE_clean'] = df_final['INTITULE_DU_POSTE'].fillna('')
df_final['Texte_clean'] = df_final['Texte_fourni'].fillna('')
df_final['Entreprise_clean'] = df_final['Entreprise'].fillna('')
df_final['Niveau_etude_clean'] = df_final["Niveau d'étude (diplome)"].fillna('')
df_final['TYPE_DE_CONTRAT_DU_POSTE_clean'] = df_final["TYPE_DE_CONTRAT_DU_POSTE"].fillna('')

#TYPE_DE_CONTRAT_DU_POSTE

# Prétraitement des données
df_final['Texte_clean'] = df_final['Texte_clean'].apply(preprocess_text)
df_final['INTITULE_DU_POSTE_clean'] = df_final['INTITULE_DU_POSTE_clean'].apply(preprocess_text)
df_final['Entreprise_clean'] = df_final['Entreprise_clean'].apply(preprocess_text)
df_final['Niveau_etude_clean'] = df_final["Niveau d'étude (diplome)"].apply(preprocess_text)   
df_final['TYPE_DE_CONTRAT_DU_POSTE_clean'] = df_final["TYPE_DE_CONTRAT_DU_POSTE"].apply(preprocess_text)

# Concaténer toutes les variables pour créer un texte combiné
df_final['Combined_text'] = df_final['INTITULE_DU_POSTE_clean']+ ' ' + df_final['Texte_clean']+ ' ' + df_final['Entreprise_clean']+ ' '+df_final['Niveau_etude_clean']+ ' '+df_final['TYPE_DE_CONTRAT_DU_POSTE_clean']

# Vectorisation des données
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_final['Combined_text'])

# Calcul de similarité
similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Seuil de similarité
threshold = 0.9

# Identification des doublons
duplicates = set()
for i in range(len(similarities)):
    for j in range(i+1, len(similarities)):
        if similarities[i,j] > threshold:
            duplicates.add(j)

# Suppression des doublons
df_final = df_final.drop(index=duplicates).reset_index(drop=True)

# Afficher ou retourner le DataFrame final
df_final


,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,Niveau,Experience_lettre,url,SITE_WEB_DE_L_ENTREPRISE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,...,NOMBRE_DE_POSTES_A_POURVOIR,TYPE_DE_CONTRAT_DU_POSTE,RAISON_SOCIALE_DE_L_ENTREPRISE,VILLE_DU_POSTE_DE_TRAVAIL,INTITULE_DU_POSTE_clean,Texte_clean,Entreprise_clean,Niveau_etude_clean,TYPE_DE_CONTRAT_DU_POSTE_clean,Combined_text
0,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,Manager / Responsable département,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,...,NaN,NaN,NaN,Abidjan,gestionnaire des ressources humaines,entreprise spécialisée dans l’ingénierie mécan...,,,,gestionnaire des ressources humaines entrepris...
1,Commercial Terrain,KOMIAN AI,Côte d'ivoire,02 Février,Débutant / Junior,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,Aujourd'hui,...,25,Consultant,NaN,,commercial terrain,komian ia est une entreprise de technologie d’...,komian ai,ts bac 2 licence lmd bac 3,consultant,commercial terrain komian ia est une entrepris...
2,Stagiaire Réseau Fibre Optique et Informatique,WANDOO TECHNOLOGIES,Côte d'ivoire,25 Avril,Stagiaire / Etudiant,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,24 Juillet,...,01,Stage – Temps partiel,NaN,,stagiaire réseau fibre optique et informatique,description de la mission participation à la r...,wandoo technologies,,stage – temps partiel,stagiaire réseau fibre optique et informatique...
3,Chiffreur BTP,Exceliam,Côte d'ivoire,08 Février,Responsable d'équipe,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,01,CDI,NaN,Abidjan,chiffreur btp,sous la responsabilité du responsable techniqu...,exceliam,master 2 ingéniorat bac 5,cdi,chiffreur btp sous la responsabilité du respon...
4,Chargé Administratif RH et Commercial Adjoint,Exceliam,Côte d'ivoire,07 Février,Responsable d'équipe,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",05 Mai,...,01,CDI| CDD,NaN,Abidjan,chargé administratif rh et commercial adjoint,sous le contrôle direct du responsable de l’ad...,exceliam,licence lmd bac 3 master 1 licence bac 4,cdi cdd,chargé administratif rh et commercial adjoint ...
5,Chargé D'Affaires Senior,Exceliam,Côte d'ivoire,08 Février,Responsable d'équipe,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,02,CDI,NaN,Abidjan,chargé daffaires senior,sous la responsabilité du responsable techniqu...,exceliam,licence lmd bac 3 master 1 licence bac 4,cdi,chargé daffaires senior sous la responsabilité...
6,Assistante Anglais et Français pour un Salon d...,FutureTech,Côte d'ivoire,04 Mars,Débutant / Junior,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juin,...,01,NaN,NaN,,assistante anglais et français pour un salon d...,dans le cadre de notre participation à un salo...,futuretech,,,assistante anglais et français pour un salon d...
7,Télévendeur H/F,OLSY GROUP,Côte d'ivoire,03 Avril,Confirmé / Expérimenté,1 à 2 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juillet,...,10,CDD,NaN,,télévendeur hf,olsy group est une entreprise qui offre des se...,olsy group,ts bac 2,cdd,télévendeur hf olsy group est une entreprise q...
8,Responsable Commercial,Exceliam,Côte d'ivoire,06 Mars,Manager / Responsable département,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte 

In [ ]:
tfidf_matrix

In [52]:
help(tfidf_matrix)

Help on csr_matrix in module scipy.sparse._csr object:

class csr_matrix(scipy.sparse._matrix.spmatrix, _csr_base)
 |  csr_matrix(arg1, shape=None, dtype=None, copy=False)
 |  
 |  Compressed Sparse Row matrix
 |  
 |  This can be instantiated in several ways:
 |      csr_array(D)
 |          with a dense matrix or rank-2 ndarray D
 |  
 |      csr_array(S)
 |          with another sparse matrix S (equivalent to S.tocsr())
 |  
 |      csr_array((M, N), [dtype])
 |          to construct an empty matrix with shape (M, N)
 |          dtype is optional, defaulting to dtype='d'.
 |  
 |      csr_array((data, (row_ind, col_ind)), [shape=(M, N)])
 |          where ``data``, ``row_ind`` and ``col_ind`` satisfy the
 |          relationship ``a[row_ind[k], col_ind[k]] = data[k]``.
 |  
 |      csr_array((data, indices, indptr), [shape=(M, N)])
 |          is the standard CSR representation where the column indices for
 |          row i are stored in ``indices[indptr[i]:indptr[i+1]]`` and their


In [11]:
similarities

array([[1.        , 0.34764986, 0.26202349, ..., 0.21265302, 0.43465652,
        0.32698244],
       [0.34764986, 1.        , 0.21974334, ..., 0.25099532, 0.34713357,
        0.31949761],
       [0.26202349, 0.21974334, 1.        , ..., 0.15843724, 0.24178448,
        0.19605545],
       ...,
       [0.21265302, 0.25099532, 0.15843724, ..., 1.        , 0.23132896,
        0.27717748],
       [0.43465652, 0.34713357, 0.24178448, ..., 0.23132896, 1.        ,
        0.32074343],
       [0.32698244, 0.31949761, 0.19605545, ..., 0.27717748, 0.32074343,
        1.        ]])

In [50]:
v.shape

(9220,)

In [41]:
similarities

array([[1.        , 0.35519365, 0.25831521, ..., 0.2113441 , 0.44008386,
        0.33125969],
       [0.35519365, 1.        , 0.21828934, ..., 0.24996177, 0.35457366,
        0.3274887 ],
       [0.25831521, 0.21828934, 1.        , ..., 0.16536667, 0.23795617,
        0.1950946 ],
       ...,
       [0.2113441 , 0.24996177, 0.16536667, ..., 1.        , 0.22464974,
        0.27218893],
       [0.44008386, 0.35457366, 0.23795617, ..., 0.22464974, 1.        ,
        0.32825131],
       [0.33125969, 0.3274887 , 0.1950946 , ..., 0.27218893, 0.32825131,
        1.        ]])

In [33]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
import urllib3
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup
import requests

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import numpy as np
import os
import time
import pandas as pd
from datetime import datetime
from selenium import webdriver
#from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By


import dateparser


def emploi_educarriere():
    # Ignorer les avertissements SSL
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
    # Fonction pour extraire le texte d'un élément HTML
    def extract_text(element, class_name=None, style=None, text_contains=None):
        if element:
            tag = element.find(class_=class_name, style=style, text=text_contains)
            return tag.text.strip() if tag else ""
        else:
            return ""

    # Fonction pour nettoyer le texte
    def clean_text(text):
        if text is not None:
            cleaned_text = text.replace('D\x92', ' ').replace('d\x92', ' ').replace('\x92', ' ').replace('\r\n', '').replace('\xa0', '')
            return cleaned_text.strip() if cleaned_text else None
        else:
            return None

    # Fonction pour extraire la date à partir d'un élément HTML
    def extract_date(element, text_contains):
        date_elements = element.find_all('a', class_='text')
        date = next((e.find('span', style='color:#FF0000;').text.strip() for e in date_elements if text_contains in e.text), "")
        return date

    # Fonction pour extraire les détails des offres d'emploi
    def scrap_job_details(urls):
        # En-tête pour éviter d'être bloqué
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }

        options = webdriver.ChromeOptions()
        #options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
        #options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
        chrome_driver_path = 'C:\\Program Files (x86)\\chromedriver.exe'
        options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Remplacez par l'emplacement réel de votre Chrome binary
        options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
        driver = webdriver.Chrome(options=options)

        # Liste pour stocker les détails de chaque emploi
        all_job_details = []

        # Parcourir les liens
        for url in urls:
            req = requests.get(url, headers=headers)
            soup = BeautifulSoup(req.text, 'html.parser')
            time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

            offres = soup.find_all('div', class_='box row')

            # Parcourir les offres d'emploi sur la page principale
            for offre in offres:
                # Trouver la balise <h4> dans la structure HTML pour extraire le lien
                offre_link_tag = offre.find('h4')

                # Vérifier si la balise <h4> a été trouvée
                if offre_link_tag:
                    # Extraire le lien de l'attribut 'href'
                    offre_link = offre_link_tag.find('a')['href']
                    all_job_details.append({'Offre_Link': offre_link})

        # Fermer le pilote Selenium à la fin
        driver.quit()

        # Concaténer tous les détails des emplois en un seul DataFrame
        if all_job_details:
            all_job_details_df = pd.DataFrame(all_job_details)
            return all_job_details_df
        else:
            print("Aucun détail d'offre d'emploi trouvé.")
            return None

    # Fonction pour récupérer les données des offres d'emploi
    def scrape_emploi_ci(url):
        try:
            response = requests.get(url, timeout=200)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Erreur de connexion à {url} : {e}")
            return pd.DataFrame()

        soup = BeautifulSoup(response.text, 'html.parser')

        job_description_wrappers = soup.find_all('div', class_='box row')

        data_list = []

        for wrapper in job_description_wrappers:
            h4_tag = wrapper.find('h4')
            poste = extract_text(h4_tag)

            entry_title_tag = wrapper.find('p', class_='entry-title')
            sous_titre = extract_text(entry_title_tag)

            a_text_tag = wrapper.find('a', class_='text')
            code = extract_text(a_text_tag, style='color:#FF0000;')

            date_edition = extract_date(wrapper, "Date d'édition:")
            date_limite = extract_date(wrapper, "Date limite:")

            pays_tag = wrapper.find('a', class_='text')
            pays = pays_tag.find_parent().text.strip().split()[-1] if pays_tag else None

            sous_titre = clean_text(sous_titre)

            data_list.append({
                'Poste': clean_text(poste),
                'Sous_titre': sous_titre,
                'Code': clean_text(code),
                'Date_DEdition': date_edition,
                'Date_limite': date_limite,
                'Pays': clean_text(pays),
                'URL': url  # Ajout de la colonne 'URL'
            })

        df = pd.DataFrame(data_list)
        return df

    # Fonction pour ajouter la colonne 'Offre_Link' à un DataFrame existant
    def add_offre_link_column(result_df):
        job_details_df = scrap_job_details(list(result_df["URL"]))
        if job_details_df is not None:
            # Join des DataFrames
            result_df = pd.concat([result_df, job_details_df], axis=1)
            return result_df
        else:
            print("Impossible d'ajouter la colonne 'Offre_Link' au DataFrame.")
            return None

    # Fonction pour extraire les informations de l'offre d'emploi
    def extract_job_information(soup, url):
        try:
            # Extraction des informations de l'offre d'emploi
            poste = soup.select_one('li.list-group-item:-soup-contains("Poste")').strong.next_sibling.strip()
            type_offre = soup.select_one('li.list-group-item:-soup-contains("Type d\'offre")').strong.next_sibling.strip()
            metiers = soup.select_one('li.list-group-item:-soup-contains("Métier(s):")').strong.next_sibling.strip()
            niveaux = soup.select_one('li.list-group-item:-soup-contains("Niveau(x):")').strong.next_sibling.strip()
            experience = soup.select_one('li.list-group-item:-soup-contains("Expérience:")').strong.next_sibling.strip()
            lieu = soup.select_one('li.list-group-item:-soup-contains("Lieu:")').strong.next_sibling.strip()
            
            # Extraction des dates de publication et de limite
            date_publication = soup.find('strong', string='Date de publication:').find_next('span').text.strip()
            date_limite = soup.find('strong', string='Date limite:').find_next('span').text.strip()
            
            # description = soup.select_one('div.text-col.post.small-post.col-md-9.col-xs-12 ul.list-group').text.strip()
            description = soup.select_one('div.entry-content').text.strip()

            return {
                "Poste": [poste],
                "Type d'offre": [type_offre],
                "Métier(s)": [metiers],
                "Niveau(x)": [niveaux],
                "Expérience": [experience],
                "Lieu": [lieu],
                "Offre_Link": [url],
                "Date de publication": [date_publication],
                "Date limite": [date_limite],
                "Description": [description]
            }
        except Exception as e:
            print(f"An error occurred while extracting job information for URL {url}: {e}")
            # Retourner un dictionnaire avec l'URL en cas d'erreur
            return {"Offre_Link": [url]}

    # Liste des liens
    urls = ["https://emploi.educarriere.ci/nos-offres?page1={}&codes=&mots_cles=&typeemploi1=&niveau1=&anciennete=&typeoffre1=&recruteur=".format(category) for category in range(1)]

    # Créer un DataFrame à partir des liens
    result_df = pd.concat([scrape_emploi_ci(url) for url in urls], ignore_index=True)

    # Ajouter la colonne 'Offre_Link'
    result_df = add_offre_link_column(result_df)
    # Supprimer les lignes avec plus de 80% de valeurs NaN
    threshold = int(result_df.shape[1] * 0.8)  # 80% des colonnes
    result_df = result_df.dropna(thresh=threshold)
    result_df = result_df.reset_index(drop=True)
    
    # Liste des URLs à scraper
    urls = list(result_df['Offre_Link'])

    # Liste pour stocker les DataFrames
    dfs = []

    # Boucle sur chaque URL
    for url in urls:
        try:
            # Envoyer une requête GET au site avec un délai de 120 secondes
            response = requests.get(url, headers=headers, verify=True, timeout=120)

            # Vérifier si la requête a réussi (statut 200)
            if response.status_code == 200:
                # Analyser le contenu de la page avec BeautifulSoup
                soup = BeautifulSoup(response.text, 'html.parser')

                # Extraire les informations sur l'emploi
                job_info = extract_job_information(soup, url)

                # Créer un DataFrame
                df = pd.DataFrame(job_info)

                # Ajouter le DataFrame à la liste
                dfs.append(df)
            else:
                print(f"Échec de la requête pour l'URL {url}. Statut : {response.status_code}")
                # Ajouter une ligne avec l'URL en cas d'erreur
                dfs.append(pd.DataFrame({"Offre_Link": [url]}))

        except requests.exceptions.Timeout:
            print(f"Timeout lors de la requête pour l'URL {url}")
            # Ajouter une ligne avec l'URL en cas d'erreur
            dfs.append(pd.DataFrame({"Offre_Link": [url]}))
        except requests.exceptions.RequestException as e:
            print(f"Une erreur s'est produite lors de la requête pour l'URL {url}: {e}")
            # Ajouter une ligne avec l'URL en cas d'erreur
            dfs.append(pd.DataFrame({"Offre_Link": [url]}))

        # Ajouter un délai de 5 secondes entre les requêtes pour éviter d'être bloqué
        time.sleep(5)

    # Concaténer tous les DataFrames en un seul DataFrame
    df_Educarriere = pd.concat(dfs, ignore_index=True)

    # Ajouter les listes existantes en tant que colonnes au DataFrame
    df_Educarriere = pd.merge(df_Educarriere, result_df, on='Offre_Link')
    df_Educarriere = df_Educarriere.drop_duplicates()
    df_Educarriere.reset_index(drop=True, inplace=True)
    
    equivalences = {
    "Poste_y": "INTITULE_DU_POSTE",
    "Type d'offre": "TYPE_DE_CONTRAT_DU_POSTE",
    "Métier(s)": "SPECIALITE",
    "Niveau(x)": "DIPLOME",
    "Expérience": "EXPERIENCE_PROFESSIONNELLE",
    "Lieu": "LIEU_DU_POSTE_DE_TRAVAIL",
    "Offre_Link": "SITE_WEB_DE_L_ENTREPRISE",
    "Date de publication": "DATE_DE_DEBUT_DE_L_OFFRE",
    "Date limite": "DATE_D_EXPIRATION_DE_L_OFFRE",
    "Poste_x": "BRANCHE_D_ACTIVITE",
    "Description": None,
    "Sous_titre": None,
    "Code": None,
    "Date_DEdition": None,
    "Date_limite": None,
    "Pays": "PAYS_DU_POSTE_DE_TRAVAIL"}
    # Fonction pour renommer les colonnes du DataFrame en conservant les colonnes sans équivalence
    def renommer_colonnes(df, equivalences):
        colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if nouvel_nom is not None}
        df_renomme = df.rename(columns=colonnes_renommees)
        return df_renomme
    
    
    # Renommer les colonnes du DataFrame
    df_Educarriere = renommer_colonnes(df_Educarriere, equivalences)
    df_Educarriere[["EXPERIENCE_PROFESSIONNELLE", 'Unite_EXPERIENCE_PROFESSIONNELLE']] = df_Educarriere["EXPERIENCE_PROFESSIONNELLE"].str.extract(r"([0-9]+)\s*([a-zA-Z]+)")


    # Réorganiser les colonnes selon vos besoins
    # Vous pouvez réorganiser les colonnes ici

    # Afficher ou retourner le DataFrame selon votre besoin
    return df_Educarriere

df_educarriere = emploi_educarriere()
df_educarriere


C:\Users\ngora\AppData\Local\Temp\ipykernel_34004\3470384900.py:45: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  tag = element.find(class_=class_name, style=style, text=text_contains)
There was an error managing chromedriver (request or response body error: operation timed out); using driver found in the cache


SSLError: HTTPSConnectionPool(host='emploi.educarriere.ci', port=443): Max retries exceeded with url: /nos-offres?page1=0&codes=&mots_cles=&typeemploi1=&niveau1=&anciennete=&typeoffre1=&recruteur= (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1006)')))